In [1]:
from dask.distributed import Client, progress

In [2]:
## this starts local workers
## if this is desired, uncomment and execute instead of the condor workers section
#client = Client(processes=False, threads_per_worker=4,
#                n_workers=1, memory_limit='2GB')

In [2]:
# this cell starts workers in condor
import os
from dask_jobqueue import HTCondorCluster
from dask import delayed
from dask.distributed import Client, as_completed

os.environ["CONDOR_CONFIG"] = "/etc/condor/condor_config"

cluster = HTCondorCluster(
    cores=1,
    memory='4gb',
    disk='1 GB',
    job_extra_directives={
       #"+SingularityImage" : '"docker://index.docker.io/coffeateam/coffea-dask"',
       "+SingularityImage" : '"docker://index.docker.io/coffeateam/coffea-dask:0.7.21-fastjet-3.4.0.1-gc3d707c"',
        "Requirements": "HasSingularityJobStart",
    }
)
cluster.adapt(minimum=1, maximum=4)


client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.HTCondorCluster
Dashboard: proxy/8787/status,
Dashboard: proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://144.92.181.248:18113,Workers: 0
Dashboard: proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
import time
import random

def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [4]:
%%time
x = inc(1)
y = dec(2)
z = add(x, y)
z

CPU times: user 28.4 ms, sys: 67.3 ms, total: 95.7 ms
Wall time: 1.2 s


3

In [5]:
import dask
inc = dask.delayed(inc)
dec = dask.delayed(dec)
add = dask.delayed(add)

In [6]:
x = inc(1)
y = dec(2)
z = add(x, y)
z

Delayed('add-91539e9c-9b32-4a23-be36-2ff3268bb68b')

In [ ]:
z.compute()

In [8]:
zs = []

In [9]:
%%time
for i in range(256):
    x = inc(i)
    y = dec(x)
    z = add(x, y)
    zs.append(z)

CPU times: user 36.8 ms, sys: 14.4 ms, total: 51.2 ms
Wall time: 44.1 ms


In [10]:
zs = dask.persist(*zs)  # trigger computation in the background


In [11]:
client.cluster.scale(10)  # ask for ten 4-thread workers

In [12]:
L = zs
while len(L) > 1:
    new_L = []
    for i in range(0, len(L), 2):
        lazy = add(L[i], L[i + 1])  # add neighbors
        new_L.append(lazy)
    L = new_L                       # swap old list for new

dask.compute(L)

([65536],)

In [15]:
# shutdown distributed.localCluster
client.shutdown()